# Yelp API - Lab



## Introduction 

Now that we've seen how the Yelp API works, it's time to put those API and SQL skills to work in order to do some basic business analysis! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to:
* Create a DB on AWS to store information from Yelp about businesses
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and insert the information into your DB
* Perform pagination to retrieve troves of data!
* Write SQL queries to answer questions about your data 

## Problem Introduction

For this lab you will analyze the yelp data for a group of businesses to learn more about an industry. You will choose a type of business (Italian Restuarants, Nail Salons, Crossfit gyms) and a location to analyze. Then you will get data from the Yelp API, store that data in a SQL Database on AWS, and write queries to answer questions about the data. 


### Process:

1. Read through the SQL questions and the API documentation to determine which pieces of information you need to pull from the Yelp API.

2. Create a DB schema with 2 tables. One for the businesses and one for the reviews.

3. Create code to:
  - Perform a search of businesses using pagination
  - Parse the API response for specific data points
  - Insert the data into your AWS DB

4. Use the functions above in a loop that will paginate over the results to retrieve all of the results. 

5. Create functions to:
  - Retrieve the reviews data of one business
  - Parse the reviews response for specific review data
  - Insert the review data into the DB

6. Using SQL, query all of the business IDs. Using the 3 Python functions you've created, run your business IDs through a loop to get the reviews for each business and insert them into your DB.

7. Write SQL queries to answer the following questions about your data.


Bonus Steps:  
- Place your helper functions in a package so that your final notebook only has the major steps listed.
- Rewrite your business search functions to be able take an argument for the type of business you are searching for.
- Add another group of businesses to your database.


## Part 1 - Understanding your data and question

Lok at the question and determine what data you will need to store in your database in order to answer the questions. Start to thin about your DB schema. What tables will you want to create and what columns will you ahve for those tables. 

Look at the API documentation, and determine what fields of the API response you will match up with the columns you want in your DB tables. 


https://www.yelp.com/developers/documentation/v3/get_started

## Part 2 - Set up the DB

Now that you are familiar with the data, create your SQL queries to create the DB and the appropriate tables. 

### Foriegn Keys

BFor this project you will have two tables that you need to link using a foreign key. Below is some information about how to create a foriegn key.  

http://www.mysqltutorial.org/mysql-foreign-key/



### Using DB:
    
For this lab, you can either store the data on one DB or put in on both of the partners DBs. If you decide to put it on one DB, you want to make sure both partners have access to it.  To do this you want to add a user to your DB.  

[how to add a new user](https://howchoo.com/g/mtm3zdq2nzv/how-to-add-a-mysql-user-and-grant-privileges)

In [1]:
## Connect to DB server on AWS
import mysql.connector
import datetime
import pandas as pd
from mysql.connector import errorcode
import config
import helplib
import requests
import json
import itertools

cnx = mysql.connector.connect(host=config.host,
                              user=config.user,
                              password=config.password)
cursor = cnx.cursor()

In [2]:
## Create new DB
q1 = 'CREATE DATABASE project1'
#cursor.execute(q1)

In [ ]:
helplib.exe_query(cursor, q1)

In [ ]:
helplib.connect_db('project1')


In [3]:
cnx = mysql.connector.connect(host=config.host,
                              user=config.user,
                              password=config.password,
                              database='project1')
cursor = cnx.cursor()

In [4]:
# Create a table for the Businesses
q2 = """CREATE TABLE businesses (
business_id varchar(200),
name varchar(50),
review_count int,
price varchar(10),
rating float
);

"""

In [5]:
# Create a table for the reviews

q3 = """CREATE TABLE reviews (
business_id varchar(200),
review_id varchar(200),
rating float,
time_created varchar(200)
text var(1000)
);

"""

In [ ]:
cursor.execute(q3)

In [ ]:
cursor.execute(q2)

## Part 3 - Create ETL pipeline for the business data from the API

Now that you know what data you need from the API, you want to write code that will execute a api call, parse those results and then insert the results into the DB.  

It is helpful to break this up into three different functions (*api call, parse results, and insert into DB*) and then you can write a function/script that pull the other three functions together. 

Let's first do this for the Business endpoint.

In [6]:
url = 'https://api.yelp.com/v3/businesses/search'
offset = 0

In [28]:
def get_data_to_sql(cursor, url, offset):
    # counter that will update the 'offset' value
    total_result = []
    #data = yelp_call(url, offset)
    for i in range(offset, 1000, 50):
        data = helplib.yelp_call(url, i)
        data_parced = helplib.parse_results(data['businesses'])
        print(type(data_parced))
        cursor.executemany(
            """INSERT INTO businesses 
        (business_id, name, review_count, price, rating) 
         VALUES (%s, %s, %s, %s, %s)""", data_parced)
        cnx.commit()

In [ ]:
get_data_to_sql(cursor, url, 0)

In [ ]:
# write a SQL query to pull back all of the business ids 
# you will need these ids to pull back the reviews for each restaurant

In [7]:
q5 = """ 
SELECT business_id FROM businesses 
"""
cursor.execute(q5)
buz_id = cursor.fetchall() 

In [8]:
len(buz_id)

1000

## Part 4 -  Create ETL pipeline for the restaurant review data from the API

You've done this for the Businesses, now you need to do this for reviews. You will follow the same process, but your functions will be specific to reviews.

In [9]:
new_url = 'https://api.yelp.com/v3/businesses/' +  buz_id[0][0] + '/reviews'
headers = {'Authorization': 'Bearer {}'.format(config.api_key), }


In [10]:
response = requests.get(new_url, headers=headers)

In [11]:
data = response.json()

In [13]:
for key in data.keys():
    print(key)
    value = response.json()[key]  #Use standard dictionary formatting
    print(type(value))  #What type is it?
    print('\n')  #Seperate out data

reviews
<class 'list'>


total
<class 'int'>


possible_languages
<class 'list'>




In [14]:
len(data['reviews'])

3

In [21]:
def get_review(list_id):
    headers = {
        'Authorization': 'Bearer {}'.format(config.api_key),
    }
    total_list = []

    for x in range(len(list_id)):
        new_url = 'https://api.yelp.com/v3/businesses/' + list_id[x][
            0] + '/reviews'
        response = requests.get(new_url, headers=headers)
        rev_2 = response.json()
        rev = rev_2['reviews']
        total_review_tuble = []

        while len(rev) < 3:
            dic = {}
            rev.append(dic)

        for i in range(len(data['reviews'])):
            rev_tup = (list_id[x][0], 
                       rev[i].get('id', None), 
                       rev[i].get('rating', None), 
                       rev[i].get('time_created',None), 
                       rev[i].get('text', None))
            total_review_tuble.append(rev_tup)

        total_list.append(tuple(total_review_tuble))
    #joining into a finla list
    total_reviews_final = list(itertools.chain.from_iterable(total_list))
    return (total_reviews_final)

In [22]:
review_data2 = get_review(buz_id)

In [24]:
type(review_data2)

list

In [25]:
len(review_data2)

3000

In [26]:
q6= """INSERT INTO reviews
    (business_id, review_id, rating, time_created, text)
    VALUES (%s, %s, %s, %s, %s)"""

In [ ]:
cursor.executemany(q6,review_data2)
cnx.commit()

## Part 5 -  Write SQL queries that will answer the questions posed. 

Now that your data is in the DB, you can answer the questions. Write SQL querires that will answer the questions. **Do not use Pandas in this lab**

In [35]:
# create connection

cnx = mysql.connector.connect(host=config.host,
                              user=config.user,
                              password=config.password,
                              database='project1')

cursor = cnx.cursor()

### SQL Questions:

Which are the 5 most reviewed businesses?

In [40]:
q7 = """
SELECT name, review_count 
FROM project1.businesses 
ORDER BY review_count DESC
LIMIT 5
;
"""
df_7 = helplib.make_df(cursor, q7)

In [41]:
df_7

,name,review_count
0,Katz's Delicatessen,11907
1,Clinton Street Baking Company,4926
2,Max Brenner - New York,4605
3,Junior's Restaurant & Bakery - 45th St.,4471
4,Serendipity 3,4137


What is the highest rating recieved in your data set and how many businesses have that rating?

In [42]:
q8 = """
SELECT name, rating 
FROM project1.businesses 
WHERE rating = 5
;
"""

In [45]:
df_8= helplib.make_df(cursor, q8)

In [46]:
df_8

,name,rating
0,Rosalu Diner,5.0
1,191 Knickerbocker,5.0
2,C&B Luncheonette,5.0
3,Yia Yia's Taverna,5.0
4,Bernie's,5.0
5,Otis,5.0
6,Lella Alimentari,5.0
7,Miss American Pie,5.0
8,La Ñapa,5.0
9,Jack the Ripper,5.0


In [47]:
len(df_8)

20

What percentage of businesses have a rating greater than or 4.5?

In [50]:
q9 = """
SELECT count(rating) 
FROM project1.businesses  
WHERE rating >= 4.5;
;
"""

In [53]:
cursor.execute(q9)
sum_rating = cursor.fetchall()

In [58]:
percentage = sum_rating[0][0]*100/1000
percentage

16.4

What percentage of businesses have a rating less than 3?

In [ ]:
q10 = """

"""

What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs?

In [ ]:
q11 = """

"""

Return the text of the reviews for the most reviewed restaurant.

In [ ]:
q12 = """

"""

Return the name of the business with the most recent review.

In [ ]:
q13 = """

"""

Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the most reviews.

In [ ]:
q14 = """

"""

Find the lowest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the least reviews.

In [23]:
q15 = """

"""

# Extra Reference help

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter and bring all of the functions together.***


***Also, something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data***


In [ ]:
def yelp_call(url_params, api_key):
    # your code to make the yelp call
    return data

In [ ]:
def parse_results(results):
    # your code to parse the result to make them easier to insert into the DB
    return parsed_results

In [ ]:
def db_insert(cnx, cursor, parsed results):
    # your code to insert and commit the results
    # I would create the connection and cursor outside of this function and then pass it through

In [ ]:
# create a variable  to keep track of which result you are in. 
cur = 0

#set up a while loop to go through and grab the result 
while cur < num and cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    db_insert(parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 50